In [ ]:
%mavenRepo s3 https://joule.s3.amazonaws.com/repo

In [ ]:
%maven software.amazon.ai:joule-api:0.2.0-SNAPSHOT
%maven org.apache.mxnet:mxnet-joule:0.2.0-SNAPSHOT
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
%maven net.java.dev.jna:jna:5.3.0
// %maven org.apache.mxnet:mxnet-native-mkl:jar:osx-x86_64:1.5.0-SNAPSHOT

In [ ]:
%%loadFromPOM
  <repositories>
    <repository>
      <id>joule</id>
      <url>https://joule.s3.amazonaws.com/repo</url>
    </repository>
  </repositories>

  <dependencies>
    <dependency>
      <groupId>org.apache.mxnet</groupId>
      <artifactId>mxnet-native-mkl</artifactId>
      <version>1.5.0-SNAPSHOT</version>
      <classifier>osx-x86_64</classifier>
    </dependency>
  </dependencies>


In [ ]:
import java.awt.image.*;
import java.nio.file.*;
import java.util.*;
import software.amazon.ai.*;
import software.amazon.ai.inference.*;
import software.amazon.ai.ndarray.*;
import software.amazon.ai.ndarray.index.*;
import software.amazon.ai.modality.*;
import software.amazon.ai.modality.cv.*;
import software.amazon.ai.util.*;


In [ ]:
var userHome = System.getProperty("user.home");
var modelDir = Paths.get(userHome, ".model_zoo/resnet50_ssd");
var imageFile = Paths.get("../examples/src/test/resources/3dogs.jpg").toRealPath();
var modelName = "resnet50_ssd_model";

modelDir

In [ ]:
var img = Images.loadImageFromFile(imageFile);
int width = img.getWidth();
int height = img.getHeight();
img

In [ ]:
"width: " + width + ", height: " + height

In [ ]:
var model = Model.loadModel(modelDir, modelName);
var synset = model.getArtifact("synset.txt", Utils::readLines);


In [ ]:
// Resize image to match the size that tained for the model

var resizedImg = Images.resizeImage(img, 512, 512);

In [ ]:
var translator = new DefaultImageTranslator();

In [ ]:
var predictor = Predictor.newInstance(model, translator);
NDList list = predictor.predict(resizedImg);

In [ ]:
// Post processing to convert NDArray to meaningful output

var detections = new ArrayList<DetectedObject>();
var array = list.get(0).get(0);
long length = array.getShape().head();
for (int i = 0; i < length; ++i) {
    try (NDArray item = array.get(i)) {
        float[] values = item.toFloatArray();
        int classId = (int) values[0];
        float probability = values[1];
        if (classId > 0 && probability > 0.2) {
            String className = synset.get(classId);

            double x = values[2];
            double y = values[3];
            double w = values[4] - x;
            double h = values[5] - y;

            var rect = new Rectangle(x, y, w, h);
            detections.add(new DetectedObject(className, probability, rect));
        }
    }
}


In [ ]:
Images.drawBoundingBox(img, detections);
img